# Exploring Data

In this notebook we're just going to explore the data without trying to do anything 'complicated'. So if we have numeric, tabular data then we will work with it like it was in Excel. If we have spatial data we will work with it to make a map. We will *not* try to link them together until the very end. 

## Downloading Data

In this section we download the data from a remote server and save it to a working directory on our computer.

### Setup

In [ ]:
from pathlib import Path

src_dir = Path.home() / Path('work/data/clean')

if not src_dir.exists():
    src_dir.mkdir(parents=True)
    print(f"Creating a 'clean' directory for data in {src_dir}.")

Below we are setting up some basic *variables* that we'll use to manage the download porcess. The data is all accessible from a CASA server called 'Orca' (`base_url`) and then we have a list of the data and geodata that we will need to download. Note that, when approaching a problem with code, we can be 'lazy' and not even specify the file extension (`.parquet`) because we can *add* then when downloading each file.

In [ ]:
base_url = 'https://orca.casa.ucl.ac.uk/~jreades/jaipur'

files = [
    'assets', 'condition', 'materials', 'occupancy', 'structure', 'utilities', 'ward_population', 'jaipur_population'
]

print(f"The tabular data files are: {', '.join(files)}")

### Download

Notice the `exists()` part -- we are checking for the existence of a file locally *before* we try to download it!

In [ ]:
# urlretrieve is a function (provided by Python) for 
# downloading a file from a URL and saving it locally
from urllib.request import urlretrieve

# For each tabular data file
for f in files:
    print(f"Retrieving {f} table.")
    save_path = Path(src_dir / f"{f}.parquet")
    if save_path.exists():
        print("\tAlready downloaded this data...")
    else: 
        print("\tDownloading...")
        urlretrieve(f"{base_url}/{f}.parquet", save_path)

## House Condition Data

Reading tables (this can include Excel, CSV, and other types of tabular data) is easily done using the `pandas` library, which we here import using the `pd` alias, so wherever you see `pd.` we are using a tool from the pandas library. The name of a table in pandas is a 'data frame', often abbreviated to `df`.

Read in the housing condition data for Jaipur. This is something that I prepared earlier using the code in the [Manage Data](./Manage_Data.ipynb) notebook. The code in that notebook is quite a bit more complex because the Census data is quite complex, but as you develop your understanding of pandas and Python you will probably find it useful!

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet(src_dir / 'condition.parquet')

### Inspecting the Data Frame

Pandas can speak to JupyterLab to produce prettily-formatted data like this:

In [ ]:
df.head(3)

List the columns:

In [ ]:
print(df.columns.to_list())

Get descriptive statistics for the data frame:

In [ ]:
df.describe()

Get descriptive statistics for only a few columns from the data frame:

In [ ]:
df[['house_condition_total_good','house_condition_total_livable', 'house_condition_total_dilapidated']].describe()

### Describing Data 

Show the first few lines for a subset of columns. 

<div class="alert alert-block alert-success">
    &#9745; <b>Notice</b> the <b>:</b> between the start and end columns (total and dilapidated below), that tells pandas it's a range!
</div>

In [ ]:
df.loc[:, 'house_condition_total_total':'house_condition_total_dilapidated'].head()

In [ ]:
df.house_condition_total_good

### Simple Plots

In [ ]:
df.house_condition_total_good.plot.hist()

Using the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.hist.html) can you work out how to change the plot so that:

- There are 20 bins.
- The plot is 70% opaque


In [ ]:
df.house_condition_total_good.plot.hist(bins=20, alpha=0.7)

In [ ]:
df.plot.hexbin(x='house_condition_total_good', y='house_condition_total_dilapidated', gridsize=20, cmap='viridis')

### Subsetting Data 

It's (potentially) interesting tha there are a few wards that are a long way off the diagonal. Is there anything special about them? Let's try to select them out of the data! We're going to do this crudely using the data itself.

In [ ]:
of_interest = df[
                    ((df.house_condition_total_good < 55) | (df.house_condition_total_dilapidated > 3))
                ].copy()
of_interest[['ward_no','area_name','house_condition_total_good','house_condition_total_dilapidated']]

## Population Data

Let's look at a different data set now just so that we can see how the *same* process works with a completely different data set.

In [ ]:
df = pd.read_parquet(src_dir / 'ward_population.parquet')
print(f"There are {df.shape[0]:,} rows and {df.shape[1]:,} columns.")
df.head()

### Changing the Data Type

In [ ]:
for c in ['Level','Name','TRU']:
    df[c] = df[c].astype('category')

df.Level.value_counts()

### Subsetting the Data

Here we see a way of using two criteria to select the rows out of a data frame:

1. We use the `Level` where the value is `WARD` AND
2. The `Name` values that start with 'Jaipur'

So *both* of these coniditions must be true for a row to be selected.

In [ ]:
# If you don't add the `copy()` to the end of this command then you 
# will get the following warning:
# A value is trying to be set on a copy of a slice from a DataFrame.
jwards = df[(df.Level=='WARD') & (df.Name.str.startswith('Jaipur'))].copy()
print(f"Have selected {jwards.shape[0]:,} wards.")

### A Joint Histogram

In [ ]:
jwards.plot.hist(column=['TOT_M','TOT_F'], bins=25, alpha=0.3)

### Data Transformation

It's hard to make much of the *unnormalised* data -- raw values don't necessarily tell us much about the relationships and differences between places. Fortunately, Pandas makes it easy to achieve that.

In [ ]:
jwards['workm_lq'] = pd.Series((df.TOT_M / df.TOT_P) / (df.TOT_M.sum() / df.TOT_P.sum()))
jwards.workm_lq.describe()

In [ ]:
jwards.workm_lq.plot.hist(bins=20)

In [ ]:
# The top 5%
print(f"The 95th percentile is {jwards.workm_lq.quantile(0.95):0.4f}")

In [ ]:
jwards[jwards.workm_lq > jwards.workm_lq.quantile(0.95)][['Name','TOT_P','TOT_M','TOT_F','workm_lq']]